### Packages

In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

### Create a DF of all the speeches

In [6]:
all_years=pd.DataFrame(columns=['link','title','speaker','event','year'])

In [8]:
# 1996 to 2006
years=range(1996,2006)
for year in years:
    speeches_one_year = pd.DataFrame()
    page = requests.get(f'https://www.federalreserve.gov/newsevents/speech/{year}speech.htm')
    soup = BeautifulSoup(page.text, 'html.parser')
    title = soup.select(".title")
    speakers = soup.select(".speaker")
    locations = soup.select(".location")
    for i in range(len(title)):
        speeches_one_year.at[i,'link'] = 'https://www.federalreserve.gov'+title[i].find_all('a', href=True)[0]['href']
        speeches_one_year.at[i,'title'] = title[i].text.split('\n')[1]
        speeches_one_year.at[i,'speaker'] = speakers[i].text.split('\n')[1].strip()
        speeches_one_year.at[i,'event'] = locations[i].text.split('\n')[1].strip()
        speeches_one_year.at[i,'year'] = year
    all_years=all_years.append(speeches_one_year,ignore_index=True)

In [12]:
# 2006 to 2021
years=range(2006,2021)
for year in years:
    if year > 2010:
        page = requests.get(f'https://www.federalreserve.gov/newsevents/speech/{year}-speeches.htm')
    else:
        page = requests.get(f'https://www.federalreserve.gov/newsevents/speech/{year}speech.htm')
    soup = BeautifulSoup(page.text, 'html.parser')
    events = soup.select(".eventlist__event")
    speeches_one_year = pd.DataFrame()
    for i,speech in enumerate(events):
        speeches_one_year.at[i,'link'] = 'https://www.federalreserve.gov'+events[i].find_all('a', href=True)[0]['href']
        speeches_one_year.at[i,'title'] = events[i].text.split('\n')[2]
        if events[i].text.split('\n')[3]=='Watch Live' or events[i].text.split('\n')[3]=='Video':
            speeches_one_year.at[i,'speaker'] = events[i].text.split('\n')[4]
            speeches_one_year.at[i,'event'] = events[i].text.split('\n')[5]
        else:
            speeches_one_year.at[i,'speaker'] = events[i].text.split('\n')[3]
            speeches_one_year.at[i,'event'] = events[i].text.split('\n')[4]
        speeches_one_year.at[i,'year'] = year
    all_years=all_years.append(speeches_one_year,ignore_index=True)